In [1]:
#import necesscary library and module
import malaya # nltk for preprocessing and classification
import tweepy # Twitter API 
import pandas as pd 
import re # Regular expression (for data preprocessing and text )
import pickle

C:\Users\MUSTAKIM\AppData\Roaming\Python\Python38\site-packages\malaya\function\__init__.py:31: UserWarning: Cannot import beam_search_ops not available for Tensorflow 2, `deep_model` for stemmer will not available to use.
  warnings.warn(


### Accessing twitter API using tweepy

In [2]:
consumer_key ="kjiUiDnn3VgEYpcQirYUWfu8f"
consumer_secret ="kYUdb0ebDtC6D92DDYvEqKShcKVqjgZ3048vyzxNprTua0MYUW"
access_token ="2301609108-l6W2GoHZUHK3ELBWSb4D4fN0J7FIbNe0BLkJQHV"
access_token_secret ="6h64rvQXSvltg9pZBUL3iuVHFKx48q0yvxQtxpWUAMY8C"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

* Now extracting tweets 
* My strategy is find the top 5 trends topic in malaysia and extract min 150 tweets each topic
* The trends should relate to either following topic(personal judgement)
  *  Political
  *  public issues
  *  crime
  *  education
  *  public figures
  *  news

* Thus,From April 20 until April 22 2021, Interesting *_#hashtag_* is as follows:
    * #KerajaanGagal (political)
    * #klcc (news, crime, places)  

In [3]:
query='#KerajaanGagal -filter:retweets' # this query will exclude retweets
post = api.search(q=query, count=10)

In [4]:
#tweets into dataframe
# dfKG stands for dataframe Kerajaan Gagal
dfKG = pd.DataFrame([tweet.text for tweet in post], columns=['Raw'])
dfKG.style

,Raw
0,"@malaysiakini Mana menteri wanita? Sibuk berkempen lagi ke, photoshoot dengan baju raya? #KerajaanGagal"
1,@SinarOnline Mimpi! #KerajaanGagal
2,@tnurhusna #KerajaanGagal
3,🤮🤮🤮 #DengkiKe #KerajaanGagal #RajaGagal #Agong https://t.co/eLGasMdu89
4,@bernamadotcom @KKMPutrajaya Bala bala bala #KerajaanGagal
5,#KerajaanGagal #DengkiKe #ApaMasalahSaudara https://t.co/AycA6xQSXu
6,Korang bangga tak dapat #KerajaanGagal https://t.co/Ipi9V7pif2
7,@malaysiakini Handled well?!!! You must be sleeping on the job!!! Wake up and get real!!! #KerajaanGagal
8,#agong #KerajaanGagal https://t.co/eDTPK7kyIg
9,#KerajaanGagal #TamatDarurat #DengkiKe latest one: #ApaMasalahSaudara welppp malaysian tengah betul betul marah… https://t.co/jc0I06mcNH


## **===============================================================================================================================================**
# **Data PreProcessing**
* 1) Preprocess using Malaya nltk preprocessing
    * english word translattion set to True
    * lowercase also set to True

* Then data convert into dataframe

* 2) Preprocess using Mus_style preprocessing
    * **Mus_style Data Preprocess** method:
        * translate emoji/emoticon to word (since some emoji/emoticon contain usefull information) using { **_dictionary is used_** }
        * remove special character ,punctuation mark, number { **_re python module is used_ **}
        * lowercasing { **_standard string operation_** }
        * (optional_still under contruction)  english word translation { **_googletrans API is used_** }

* Then clean data convert and merge into existing dataframe

In [5]:
#initialize malaya preprocess model
mPP = malaya.preprocessing.preprocessing(translate_english_to_bm=True, lowercase=True)

In [6]:
def preProcessMalaya(text):
  return ' '.join(mPP.process(text))

In [7]:
dfKG['cleanData_Malaya'] = dfKG['Raw'].astype(str).apply(preProcessMalaya)

In [8]:
dfKG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Raw               10 non-null     object
 1   cleanData_Malaya  10 non-null     object
dtypes: object(2)
memory usage: 288.0+ bytes


#### **-----------------------------------------------------------------------------------------------------------------------------------------------**
### <-- End of Malaya Preprocessing 
### Mus_style Preprocessing start -->

In [9]:
import googletrans as gt
from googletrans import Translator

In [10]:
print(gt.LANGUAGES)#check available languages

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [11]:
text1 ='good morning everybody'
text2 ='Apa khabar awak?'
text3 ='안녕하세요'
text4 = 'Apa khabar awak?.Im fine terima kasih'


In [12]:
translator = Translator()

In [13]:
# just checking the translator
print(translator.detect(text1))
print(translator.detect(text2))
print(translator.detect(text3))
print(translator.detect(text4))
print(translator.translate(text1, dest='ms'))
print(translator.translate(text2, dest='ko'))
print(translator.translate(text3, dest='en'))

Detected(lang=en, confidence=None)
Detected(lang=ms, confidence=None)
Detected(lang=ko, confidence=None)
Detected(lang=ms, confidence=None)
Translated(src=en, dest=ms, text=selamat Pagi semua, pronunciation=None, extra_data="{'confiden...")
Translated(src=ms, dest=ko, text=잘 있었 니?, pronunciation=jal iss-eoss ni?, extra_data="{'confiden...")
Translated(src=ko, dest=en, text=Good morning, pronunciation=None, extra_data="{'confiden...")


In [14]:
 #Translate english word to bm
    # the steps as as follow:
    # 1)split the string into words
    # 2) detect words language 
    #     *if en --> covert
    #     * else -->skip
    # 3) append the string
    
def trans_En_to_Bm(text):
  sent2Word = text.split() #step1:splitting/ tekenizing the sentence
  # sentSize = len(sent2Word)
  # print(sentSize)
  # convText=[]*  sentSize
   convText=[]
  for word in sent2Word:
    lang = translator.detect(word)# step2)-1 :detect language
    if (lang.lang =='en'):
      word = translator.translate(word, dest='ms') #step2)-2 :translate english to bahasa melayu
      convText.append(word.text)
    else:
      convText.append(word)

  text = ' '.join(convText)
  print(text)
  return text

In [15]:
# removing emoji
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [16]:
with open('../Twitter SA_Malay lang/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emoj in Emoji_Dict:
        text = re.sub(r'('+emoj+')', "_".join(Emoji_Dict[emoj].replace(",","").replace(":","").split()), text)
    return text

In [17]:
text = "I won 🥇 in 🏏"
text2 ='🐢 eat 🍄 and drive 🚁'
text3 ='sedaq diri sikit heh 😏 Kenapa'

convert_emojis_to_word(text3)

'sedaq diri sikit heh smirking_face Kenapa'

In [20]:
#define class for re preperocessing

def preProcessMus(text):
  #1st step <---- remove emoji and emoticon
  text = convert_emojis_to_word(text)
  text =' '.join(text.split('_')) # split word that merge by underscore '_'
  text = re.sub(r'https?:\/\/\S+', '', text)#remove hyperlink

  #4th step<--- translate english word to bahasa
  text = trans_En_to_Bm(text) ### translate english word to bahasa melayu


  #2nd step<--- remove special character and number
  # after ....re.sub(... the single 'r' stands for raw string. 
  #so it will ignore all the escape character python has. 
  #as for example '\n' which refer as new line. it will ignore
  text=re.sub('/\b\S\s\b/','', text)
  text= re.sub(r'@[A-Za-z0-9_A-Za-z0-9]+', '', text)#remove @mentions or user
  text = re.sub(r'#', '',text)#remove #hashtag 
  text= re.sub(r'RT[\s]+', '', text)#remove retweet
  text=re.sub(r'[-()\"''#!@$%^&*{}?.,:]', '',text)#remove speial character and punctuation


  #3rd step<--- first lowercase the text
  text = text.lower()  

  return text

In [21]:
#testing my Preprocessing algo

text5 = '@Marina_Ibrahims Kebabian PN makin terserlah dan muhyiddin kimak\n#KerajaanGagal'
text6 = '@kuasasiswa you made my day!👏😂\nso far the BEST troll ever!\nso much effort you put in👍\n#Keraja...'
text7 = 'what is the problem? hari-hari nak makan 🍿'
text8 = '@DenaBahrin Kompaun mungkin dikenakan kepada pihak #kerajaanGagal, jika pkp terpaksa dilaksanakan kerana wujud kluster ba… https://t.co/RxXc5iGyMV'

print (preProcessMus(text5))
print(preProcessMus(text6))
print(preProcessMus(text7))
print(preProcessMus(text8))

10
@Marina. Ibrahims Kebabian Pn. makin. terserlah dan muhyiddin kimak #KerajaanGagal
 ibrahims kebabian pn makin terserlah dan muhyiddin kimak kerajaangagal
24
@kuasasiswa anda dibuat my hari! Handsface. dengan air mata of. Joy. So. jauh the Best. troll. Pernah! So. banyak usaha anda letak inthumbs. up. #Keraja...
 anda dibuat my hari handsface dengan air mata of joy so jauh the best troll pernah so banyak usaha anda letak inthumbs up keraja
8
apa adalah the Masalah? hari-hari nak makan popcorn.
apa adalah the masalah harihari nak makan popcorn
15
@DenaBahrin Kompaun mungkin dikenakan kepada pihak #kerajaanGagal, jika pkp terpaksa dilaksanakan kerana wujud kluster ba…
 kompaun mungkin dikenakan kepada pihak kerajaangagal jika pkp terpaksa dilaksanakan kerana wujud kluster ba…


In [134]:
#dfKG['cleanData_Malaya'] = dfKG['Raw'].astype(str).apply(preProcessMalaya)
dfKG['cleanData-MUS'] = dfKG['Raw'].astype(str).apply(preProcessMus)

10
malam pun panas yarabbi #KerajaanGagal baru bulan FaceWew. bulan muka
14
@apanakdikato Kertas untuk buka siasatan belum dibuat di kilang Kertas lagi. Kena tunggu. #KerajaanGagal
13
Menarik, menarik bulan. Kita telah : #KerajaanGagal #TamatDarurat #DengkiKe Dan of. kursus, @KKMPutrajaya…
15
@DenaBahrin Kompaun mungkin dikenakan kepada pihak kerajaan, jika pkp terpaksa dilaksanakan kerana wujud kluster ba…
10
#KerajaanGagal #Pdrm Jangan kata nak buka kertas siasatan pulak eh.
1
#KerajaanGagal
2
@SinarOnline #KerajaanGagal
13
@ReeNGaston #KerajaanGagal masih ada muka nak kata "urus dengan baik" #ApaMasalahSaudara kan? #DengkiKe
19
@MuhyiddinYassin Sudahlah Mahiaddin. Rakyat dh ramai sedar kau PM. PNipu. Kau tabur je la apa pon..org dh muak.. #KerajaanGagal
12
Mengaku je la #KerajaanGagal Mana ada benda yg turun, semua naik je.


In [135]:
dfKG.style

,Raw,cleanData_Malaya,cleanData-MUS
0,malam pun panas yarabbi #KerajaanGagal 🌚🌚,malam pun panas yarabbi kerajaangagal 🌚 🌚,malam pun panas yarabbi baru bulan facewew bulan muka
1,@apanakdikato Kertas untuk buka siasatan belum dibuat di kilang Kertas lagi. Kena tunggu. #KerajaanGagal,kertas untuk buka siasatan belum dibuat di kilang kertas lagi . kena tunggu . kerajaangagal,kertas untuk buka siasatan belum dibuat di kilang kertas lagi kena tunggu
2,"Interesting, interesting month. We have : #KerajaanGagal #TamatDarurat #DengkiKe And of course, @KKMPutrajaya… https://t.co/WNCg3HCQvK","menarik , menarik bulan . kami ada : kerajaangagal tamatdarurat dengkike dan daripada kursus , …",menarik menarik bulan kita telah dan of kursus …
3,"@DenaBahrin Kompaun mungkin dikenakan kepada pihak kerajaan, jika pkp terpaksa dilaksanakan kerana wujud kluster ba… https://t.co/RxXc5iGyMV","kompaun mungkin dikenakan kepada pihak kerajaan , jika pkp terpaksa dilaksanakan kerana wujud kluster ba …",kompaun mungkin dikenakan kepada pihak kerajaan jika pkp terpaksa dilaksanakan kerana wujud kluster ba…
4,#KerajaanGagal #Pdrm Jangan kata nak buka kertas siasatan pulak eh. https://t.co/S4kAkb3c3u,kerajaangagal pdrm jangan kata nak buka kertas siasatan pula eh .,jangan kata nak buka kertas siasatan pulak eh
5,#KerajaanGagal https://t.co/1I58Lim4px,kerajaangagal,
6,@SinarOnline #KerajaanGagal,kerajaangagal,
7,"@ReeNGaston #KerajaanGagal masih ada muka nak kata ""urus dengan baik"" #ApaMasalahSaudara kan? #DengkiKe","kerajaangagal masih ada muka nak kata "" urus dengan baik "" apamasalahsaudara kan ? dengkike",masih ada muka nak kata urus dengan baik kan
8,@MuhyiddinYassin Sudahlah Mahiaddin. Rakyat dh ramai sedar kau PM PNipu. Kau tabur je la apa pon..org dh muak.. #KerajaanGagal,sudahlah mahiaddin . rakyat dah ramai sedar kamu pm pnipu . kamu tabur sahaja lah apa pun . orang dah muak . kerajaangagal,sudahlah mahiaddin rakyat dh ramai sedar kau pm pnipu kau tabur je la apa ponorg dh muak
9,"Admit je la #KerajaanGagal Mana ada benda yg turun, semua naik je. https://t.co/ljBNGZ71VD","mengaku sahaja lah kerajaangagal mana ada benda yang turun , semua naik sahaja .",mengaku je la mana ada benda yg turun semua naik je
